In [57]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pandas as pd
import os, glob
import seaborn as sns
from scipy.stats import kruskal
import scikit_posthocs as sp
from scipy.stats import mannwhitneyu
from dotenv import load_dotenv

In [59]:
load_dotenv() 

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/fa/.config/gcloud/application_default_credentials.json'
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101'
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()
from google.cloud import bigquery_storage_v1

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [60]:
%%bigquery Train_set_df
select * from  `som-nero-phi-jonc101.PEDsblood_culture_stewardship.cohort` where Extract(year from blood_culture_order_datetime)<2022

Query is running:   0%|          |

Downloading:   0%|          |

In [61]:
Train_set_df[(Train_set_df.positive_blood_culture==1)|(Train_set_df.positive_blood_culture_in_week==1)][['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

(856, 3)

In [62]:
Train_set_df.columns.values

array(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded',
       'blood_culture_order_datetime', 'birth_date_jittered', 'bmi',
       'positive_blood_culture', 'positive_blood_culture_in_week',
       'earliest_iv_antibiotic_datetime', 'min_heartrate',
       'max_heartrate', 'avg_heartrate', 'median_heartrate',
       'min_resprate', 'max_resprate', 'avg_resprate', 'median_resprate',
       'min_temp', 'max_temp', 'avg_temp', 'median_temp', 'min_sysbp',
       'max_sysbp', 'avg_sysbp', 'median_sysbp', 'min_diasbp',
       'max_diasbp', 'avg_diasbp', 'median_diasbp', 'min_wbc', 'max_wbc',
       'avg_wbc', 'median_wbc', 'min_neutrophils', 'max_neutrophils',
       'avg_neutrophils', 'median_neutrophils', 'min_lymphocytes',
       'max_lymphocytes', 'avg_lymphocytes', 'median_lymphocytes',
       'min_hgb', 'max_hgb', 'avg_hgb', 'median_hgb', 'min_plt',
       'max_plt', 'avg_plt', 'median_plt', 'min_na', 'max_na', 'avg_na',
       'median_na', 'min_hco3', 'max_hco3', 'avg_hco3',

In [63]:
Train_set_df[(Train_set_df.positive_blood_culture==1)|(Train_set_df.positive_blood_culture_in_week==1)][['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

(856, 3)

In [64]:
Train_set_df[(Train_set_df.positive_blood_culture==0)|(Train_set_df.positive_blood_culture_in_week==0)][['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

(19721, 3)

In [65]:
%%bigquery Val_set_df
select * from `som-nero-phi-jonc101.PEDsblood_culture_stewardship.cohort` 
where  Extract(year from blood_culture_order_datetime)>=2022 and 
Extract(year from blood_culture_order_datetime)<2023

Query is running:   0%|          |

Downloading:   0%|          |

In [66]:
%%bigquery Test_set_df
select * from `som-nero-phi-jonc101.PEDsblood_culture_stewardship.cohort`
where Extract(year from blood_culture_order_datetime)>=2023 

Query is running:   0%|          |

Downloading:   0%|          |

In [67]:
missing_percentage = Test_set_df.isnull().mean() * 100
missing_percentage[missing_percentage>80]

earliest_iv_antibiotic_datetime     91.617580
min_lactate                         82.182450
max_lactate                         82.182450
avg_lactate                         82.182450
median_lactate                      82.182450
infective_endocarditis              95.937170
septic_thrombophlebitis            100.000000
vascular_graft_infection            94.390575
CRBSI                               84.256155
infectious_discitis                100.000000
epidural_abscess                    89.309772
septic_arthritis                    99.954690
meningitis                          99.758345
meningitis_bacteria                 99.873131
cholangitis                         94.076424
Line_Presense                       97.806978
min_anc                             99.915421
max_anc                             99.915421
avg_anc                             99.915421
median_anc                          99.915421
min_alc                             99.915421
max_alc                           

# SIRS Criteria 

In [68]:
Train_set_df['Label'] = (Train_set_df['positive_blood_culture'] | Train_set_df['positive_blood_culture_in_week'])
Test_set_df['Label'] = (Test_set_df['positive_blood_culture'] | Test_set_df['positive_blood_culture_in_week'])
Val_set_df['Label'] = (Val_set_df['positive_blood_culture'] | Val_set_df['positive_blood_culture_in_week'])


Train_set_df['birth_date_jittered'] = pd.to_datetime(Train_set_df['birth_date_jittered'])
Train_set_df['blood_culture_order_datetime'] = pd.to_datetime(Train_set_df['blood_culture_order_datetime'])
Train_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Train_set_df['earliest_iv_antibiotic_datetime'])

Val_set_df['birth_date_jittered'] = pd.to_datetime(Val_set_df['birth_date_jittered'])
Val_set_df['blood_culture_order_datetime'] = pd.to_datetime(Val_set_df['blood_culture_order_datetime'])
Val_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Val_set_df['earliest_iv_antibiotic_datetime'])


Test_set_df['birth_date_jittered'] = pd.to_datetime(Test_set_df['birth_date_jittered'])
Test_set_df['blood_culture_order_datetime'] = pd.to_datetime(Test_set_df['blood_culture_order_datetime'])
Test_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Test_set_df['earliest_iv_antibiotic_datetime'])


# Calculate the difference in hours:
Train_set_df['age'] = (Train_set_df['blood_culture_order_datetime'] - Train_set_df['birth_date_jittered']).dt.days 
Test_set_df['age'] = (Test_set_df['blood_culture_order_datetime'] - Test_set_df['birth_date_jittered']).dt.days #/ 3600
Val_set_df['age'] = (Val_set_df['blood_culture_order_datetime'] - Val_set_df['birth_date_jittered']).dt.days #/ 3600

Train_set_df['hours_between_cult_abx'] = (Train_set_df['blood_culture_order_datetime'] - Train_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600
Test_set_df['hours_between_cult_abx'] = (Test_set_df['blood_culture_order_datetime'] - Test_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600
Val_set_df['hours_between_cult_abx'] = (Val_set_df['blood_culture_order_datetime'] - Val_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600


In [69]:
group_cols = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']
Test_set_df['sirs_temp'] = (Test_set_df['max_temp'] > 38) | (Test_set_df['min_temp'] < 36)

In [70]:
# SIRS HR
Test_set_df['sirs_hr'] = 0
Test_set_df.loc[
    (
        (Test_set_df['max_heartrate'] > 180) | (Test_set_df['min_heartrate'] < 100)
    ) & (
        (Test_set_df['age'] >= 30) & (Test_set_df['age'] <= 365)
    ),
    'sirs_hr'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_heartrate'] > 140)
    ) & (
        (Test_set_df['age'] > 365) & (Test_set_df['age'] <= (365*2) )
    ),
    'sirs_hr'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_heartrate'] > 120)
    ) & (
        (Test_set_df['age'] > (2*365)) & (Test_set_df['age'] <= (365*5) )
    ),
    'sirs_hr'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_heartrate'] > 118)
    ) & (
        (Test_set_df['age'] > (5*365)) & (Test_set_df['age'] <= (365*9) )
    ),
    'sirs_hr'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_heartrate'] > 100)
    ) & (
        (Test_set_df['age'] > (9*365)) & (Test_set_df['age'] <= (365*15) )
    ),
    'sirs_hr'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_heartrate'] > 90)
    ) & (
        (Test_set_df['age'] > (15*365))
    ),
    'sirs_hr'
] = 1




In [71]:
# SIRS RR
Test_set_df['sirs_rr'] = 0

Test_set_df.loc[
    (
        (Test_set_df['max_resprate'] > 53)
    ) & (
        (Test_set_df['age'] >= 30) & (Test_set_df['age'] <= 365)
    ),
    'sirs_rr'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_resprate'] > 37)
    ) & (
        (Test_set_df['age'] > 365) & (Test_set_df['age'] <= (365*2) )
    ),
    'sirs_rr'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_resprate'] > 28)
    ) & (
        (Test_set_df['age'] > (2*365)) & (Test_set_df['age'] <= (365*5) )
    ),
    'sirs_rr'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_resprate'] > 25)
    ) & (
        (Test_set_df['age'] > (5*365)) & (Test_set_df['age'] <= (365*9) )
    ),
    'sirs_rr'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_resprate'] > 20)
    ) & (
        (Test_set_df['age'] > (9*365)) & (Test_set_df['age'] <= (365*15) )
    ),
    'sirs_rr'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_resprate'] > 20)
    ) & (
        (Test_set_df['age'] > (15*365))
    ),
    'sirs_rr'
] = 1

In [72]:
Test_set_df['sirs_wbc'] = 0

Test_set_df.loc[
    (
        (Test_set_df['max_wbc'] > 17.5) & (Test_set_df['min_wbc'] < 5.5)
    ) & (
        (Test_set_df['age'] >= 30) & (Test_set_df['age'] <= 365)
    ),
    'sirs_wbc'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_wbc'] > 17) & (Test_set_df['min_wbc'] < 6)
    ) & (
        (Test_set_df['age'] > 365) & (Test_set_df['age'] <= (365*2) )
    ),
    'sirs_wbc'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_wbc'] > 15.5)  & (Test_set_df['min_wbc'] < 5.5)
    ) & (
        (Test_set_df['age'] > (2*365)) & (Test_set_df['age'] <= (365*5) )
    ),
    'sirs_wbc'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_wbc'] > 13.5) & (Test_set_df['min_wbc'] < 4.5)
    ) & (
        (Test_set_df['age'] > (5*365)) & (Test_set_df['age'] <= (365*9) )
    ),
    'sirs_wbc'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_wbc'] > 13.5)& (Test_set_df['min_wbc'] < 4.5)
    ) & (
        (Test_set_df['age'] > (9*365)) & (Test_set_df['age'] <= (365*12) )
    ),
    'sirs_wbc'
] = 1


Test_set_df.loc[
    (
        (Test_set_df['max_wbc'] > 13) & (Test_set_df['min_wbc'] < 4.5)
    ) & (
        (Test_set_df['age'] > (12*365)) & (Test_set_df['age'] <= (365*15) )
    ),
    'sirs_wbc'
] = 1

Test_set_df.loc[
    (
        (Test_set_df['max_wbc'] > 12) & (Test_set_df['min_wbc'] < 4) 
    ) & (
        (Test_set_df['age'] > (15*365))
    ),
    'sirs_wbc'
] = 1

In [81]:
# Sum up SIRS criteria to get a score from 0–4
Test_set_df['sirs_score'] = Test_set_df[['sirs_temp', 'sirs_hr', 'sirs_rr', 'sirs_wbc']].sum(axis=1)

# Optional: create a binary flag for meeting at least 2 criteria and either temp or wbc positive
Test_set_df['sirs_2plus_temp_or_wbc'] = (
    (Test_set_df['sirs_temp'] | Test_set_df['sirs_wbc']) &
    (Test_set_df['sirs_score'] >= 2))

Test_set_df['sirs_2plus'] = (
    (Test_set_df['sirs_score'] >= 3))
# we should have temp or  wbc to habe hr and rr be counted. 

In [82]:
# Step 1: AUC
auc = roc_auc_score(Test_set_df['Label'], Test_set_df['sirs_2plus'])
print(f"AUC for SIRS score: {auc:.3f}")

# Step 2: ROC
fpr, tpr, thresholds = roc_curve(Test_set_df['Label'], Test_set_df['sirs_2plus'])

# Step 3: Find threshold for ~80% sensitivity
target_sens = 0.80
idx = (np.abs(tpr - target_sens)).argmin()
best_threshold = thresholds[idx]
print(f"Threshold for ~80% sensitivity: {best_threshold:.3f}")

# Step 4: Binarize
y_pred = (Test_set_df['sirs_2plus'] >= best_threshold).astype(int)

# Step 5: Confusion matrix (✅ now fixed)
tn, fp, fn, tp = confusion_matrix(Test_set_df['Label'], y_pred).ravel()

# Step 6: Metrics
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
npv = tn / (tn + fn) if (tn + fn) > 0 else 0

print(f"Sensitivity: {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")
print(f"PPV: {ppv:.3f}")
print(f"NPV: {npv:.3f}")

AUC for SIRS score: 0.581
Threshold for ~80% sensitivity: 1.000
Sensitivity: 0.625
Specificity: 0.536
PPV: 0.015
NPV: 0.992


In [76]:
#95% CI

import numpy as np
from sklearn.metrics import roc_auc_score

def bootstrap_auc_ci(y_true, y_score, n_bootstraps=100, ci=0.95, seed=42):
    rng = np.random.RandomState(seed)
    bootstrapped_scores = []

    for _ in range(n_bootstraps):
        # sample with replacement
        indices = rng.choice(np.arange(len(y_true)), size=len(y_true), replace=True)
        if len(np.unique(y_true[indices])) < 2:
            continue  # skip if no positive or negative cases
        score = roc_auc_score(y_true[indices], y_score[indices])
        bootstrapped_scores.append(score)

    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()

    # Compute lower and upper bound of CI
    lower_bound = np.percentile(sorted_scores, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(sorted_scores, (1 + ci) / 2 * 100)

    return roc_auc_score(y_true, y_score), (lower_bound, upper_bound)

y_true = Test_set_df['Label'].values
y_score = Test_set_df['sirs_2plus_temp_or_wbc'].values

auc, (ci_lower, ci_upper) = bootstrap_auc_ci(y_true, y_score)
print(f"AUC: {auc:.3f} (95% CI: {ci_lower:.3f}–{ci_upper:.3f})")


AUC: 0.529 (95% CI: 0.519–0.541)


In [67]:
from sklearn.metrics import confusion_matrix, roc_curve, precision_score, recall_score

y_true = Test_set_df['Label']
y_scores = Test_set_df['sirs_score']  # Or predicted probabilities from a model


# Get fpr, tpr, thresholds
fpr, tpr, thresholds = roc_curve(y_true, y_scores)

# Find threshold for 90% sensitivity
target_sensitivity = 0.75
idx = (np.abs(tpr - target_sensitivity)).argmin()
best_threshold = thresholds[idx]

# Apply threshold to get binary predictions
y_pred = (y_scores >= best_threshold).astype(int)

# Confusion matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

# Calculate metrics
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

print(f"Threshold for ~90% Sensitivity: {best_threshold:.2f}")
print(f"Sensitivity: {tpr[idx]:.3f}")
print(f"Specificity: {specificity:.3f}")
print(f"PPV: {ppv:.3f}")
print(f"NPV: {npv:.3f}")

Threshold for ~90% Sensitivity: 3.00
Sensitivity: 0.921
Specificity: 0.092
PPV: 0.012
NPV: 0.990


# Data Prepration

In [47]:
Train_set_df['birth_date_jittered'] = pd.to_datetime(Train_set_df['birth_date_jittered'])
Train_set_df['blood_culture_order_datetime'] = pd.to_datetime(Train_set_df['blood_culture_order_datetime'])
Train_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Train_set_df['earliest_iv_antibiotic_datetime'])

Val_set_df['birth_date_jittered'] = pd.to_datetime(Val_set_df['birth_date_jittered'])
Val_set_df['blood_culture_order_datetime'] = pd.to_datetime(Val_set_df['blood_culture_order_datetime'])
Val_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Val_set_df['earliest_iv_antibiotic_datetime'])


Test_set_df['birth_date_jittered'] = pd.to_datetime(Test_set_df['birth_date_jittered'])
Test_set_df['blood_culture_order_datetime'] = pd.to_datetime(Test_set_df['blood_culture_order_datetime'])
Test_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Test_set_df['earliest_iv_antibiotic_datetime'])


# Calculate the difference in hours:
Train_set_df['age'] = (Train_set_df['blood_culture_order_datetime'] - Train_set_df['birth_date_jittered']).dt.days 
Test_set_df['age'] = (Test_set_df['blood_culture_order_datetime'] - Test_set_df['birth_date_jittered']).dt.days #/ 3600
Val_set_df['age'] = (Val_set_df['blood_culture_order_datetime'] - Val_set_df['birth_date_jittered']).dt.days #/ 3600

Train_set_df['hours_between_cult_abx'] = (Train_set_df['blood_culture_order_datetime'] - Train_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600
Test_set_df['hours_between_cult_abx'] = (Test_set_df['blood_culture_order_datetime'] - Test_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600
Val_set_df['hours_between_cult_abx'] = (Val_set_df['blood_culture_order_datetime'] - Val_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600


In [48]:
Train_set_df.columns.values

array(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded',
       'blood_culture_order_datetime', 'birth_date_jittered', 'bmi',
       'positive_blood_culture', 'positive_blood_culture_in_week',
       'earliest_iv_antibiotic_datetime', 'min_heartrate',
       'max_heartrate', 'avg_heartrate', 'median_heartrate',
       'min_resprate', 'max_resprate', 'avg_resprate', 'median_resprate',
       'min_temp', 'max_temp', 'avg_temp', 'median_temp', 'min_sysbp',
       'max_sysbp', 'avg_sysbp', 'median_sysbp', 'min_diasbp',
       'max_diasbp', 'avg_diasbp', 'median_diasbp', 'min_wbc', 'max_wbc',
       'avg_wbc', 'median_wbc', 'min_neutrophils', 'max_neutrophils',
       'avg_neutrophils', 'median_neutrophils', 'min_lymphocytes',
       'max_lymphocytes', 'avg_lymphocytes', 'median_lymphocytes',
       'min_hgb', 'max_hgb', 'avg_hgb', 'median_hgb', 'min_plt',
       'max_plt', 'avg_plt', 'median_plt', 'min_na', 'max_na', 'avg_na',
       'median_na', 'min_hco3', 'max_hco3', 'avg_hco3',

# Feature Set

In [17]:
Identifiers=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']
Labels=['positive_blood_culture','positive_blood_culture_in_week']
Labs=['min_heartrate','max_heartrate', 'avg_heartrate', 'median_heartrate',
       'min_resprate', 'max_resprate', 'avg_resprate', 'median_resprate',
       'min_temp', 'max_temp', 'avg_temp', 'median_temp', 'min_sysbp',
       'max_sysbp', 'avg_sysbp', 'median_sysbp', 'min_diasbp',
       'max_diasbp', 'avg_diasbp', 'median_diasbp', 'min_wbc', 'max_wbc',
       'avg_wbc', 'median_wbc', 'min_neutrophils', 'max_neutrophils',
       'avg_neutrophils', 'median_neutrophils', 'min_lymphocytes',
       'max_lymphocytes', 'avg_lymphocytes', 'median_lymphocytes',
       'min_hgb', 'max_hgb', 'avg_hgb', 'median_hgb', 'min_plt',
       'max_plt', 'avg_plt', 'median_plt', 'min_na', 'max_na', 'avg_na',
       'median_na', 'min_hco3', 'max_hco3', 'avg_hco3', 'median_hco3',
       'min_bun', 'max_bun', 'avg_bun', 'median_bun', 'min_cr', 'max_cr',
       'avg_cr', 'median_cr', 'min_lactate', 'max_lactate', 'avg_lactate',
       'median_lactate', 'min_procalcitonin', 'max_procalcitonin',
       'avg_procalcitonin', 'median_procalcitonin','Leukocyte_Esterase',
       'WBC_urine', 'Bacteria_urine','Nitrite_urine', 
      'min_anc', 'max_anc', 'avg_anc', 'median_anc',
       'min_alc', 'max_alc', 'avg_alc', 'median_alc', 'min_spo2',
       'max_spo2', 'avg_spo2', 'median_spo2']
Demos=[ 'gender','age','bmi']
ABX=['vanc', 'zosyn', 'vanc_zosyn', 'other_ABX']
Time_Varient_features=['hours_between_cult_abx']
Diagnosis= ['Line_Presense','Transplant'] #bacteremia', 'septic_shock', 'infective_endocarditis',
      # 'septic_thrombophlebitis', 'vascular_graft_infection', 'CRBSI',
      # 'infectious_discitis', 'epidural_abscess', 'septic_arthritis',
      # 'meningitis', 'meningitis_bacteria', 'cholangitis',
      # 'bacterial_cholangitis', 'pyelonephritis',
      # 'acute_bacterial_pyelonephritis', 'severe_pneumonia',
      # 'acute_hematogenous_osteomyelitis', 'asplenia',
      # 'immunocompromised_state', 'severe_cellulitis', 'cystitis',
      # 'prostatitis', 'CAP', 'diabetic_foot_infection', 'colitis',
      # 'aspiration_pneumonia', 'uncomplicated_cholecystitis',
      # 'uncomplicated_diverticulitis', 'Uncomplicated_pancreatitis',
Feature_set=Identifiers+Labels+Labs+Demos+Diagnosis#+Time_Varient_features+ABX
Train_set_df=Train_set_df[Feature_set]
Test_set_df=Test_set_df[Feature_set]
Val_set_df=Val_set_df[Feature_set]
                       

In [18]:
Train_set_df.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'],inplace=True)
Val_set_df.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'],inplace=True)
Test_set_df.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'],inplace=True)

In [19]:
Train_set_df['datapoint'] = Train_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1
Test_set_df['datapoint'] = Train_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1
Val_set_df['datapoint'] = Val_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1


In [20]:
Train_set_df.Bacteria_urine.unique()

array([None, 'NEGATIVE', 'POSITIVE'], dtype=object)

In [21]:
Train_set_df['gender'] = Train_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))
Test_set_df['gender'] = Test_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))
Val_set_df['gender'] = Val_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))


Train_set_df['Leukocyte_Esterase'] = Train_set_df['Leukocyte_Esterase'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Test_set_df['Leukocyte_Esterase'] = Test_set_df['Leukocyte_Esterase'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Val_set_df['Leukocyte_Esterase'] = Val_set_df['Leukocyte_Esterase'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))


Train_set_df['WBC_urine'] = Train_set_df['WBC_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Test_set_df['WBC_urine'] = Test_set_df['WBC_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Val_set_df['WBC_urine'] = Val_set_df['WBC_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))


Train_set_df['Bacteria_urine'] = Train_set_df['Bacteria_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Test_set_df['Bacteria_urine'] = Test_set_df['Bacteria_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Val_set_df['Bacteria_urine'] = Val_set_df['Bacteria_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))


Train_set_df['Nitrite_urine'] = Train_set_df['Nitrite_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Test_set_df['Nitrite_urine'] = Test_set_df['Nitrite_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Val_set_df['Nitrite_urine'] = Val_set_df['Nitrite_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))


Train_set_df['ET_Tube'] = Train_set_df['Line_Presense'].apply(lambda x: 1 if x == 'ET_Tube' else 0)
Test_set_df['ET_Tube'] = Test_set_df['Line_Presense'].apply(lambda x: 1 if x == 'ET_Tube' else 0 )
Val_set_df['ET_Tube'] = Val_set_df['Line_Presense'].apply(lambda x: 1 if x == 'ET_Tube' else 0)



Train_set_df['otherline'] = Train_set_df['Line_Presense'].apply(lambda x: 1 if x == 'otherline' else 0)
Test_set_df['otherline'] = Test_set_df['Line_Presense'].apply(lambda x: 1 if x == 'otherline' else 0 )
Val_set_df['otherline'] = Val_set_df['Line_Presense'].apply(lambda x: 1 if x == 'otherline' else 0)


In [22]:
Train_set_df.shape[0]+Test_set_df.shape[0]+Val_set_df.shape[0]

26829

# Logistic Regression 

In [190]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from fancyimpute import IterativeImputer as FancyIterativeImputer

In [192]:
selected_features = [
    'bmi', 'avg_neutrophils', 'age', 'max_na', 'max_heartrate', 'median_procalcitonin', 
    'min_temp', 'avg_lymphocytes', 'min_diasbp', 'avg_spo2', 'min_resprate', 'max_hco3', 
    'min_wbc', 'min_hgb', 'median_bun', 'max_cr', 'ET_Tube', 'WBC_urine', 'Bacteria_urine', 'Leukocyte_Esterase',
    'positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded',
    'datapoint','Line_Presense']
Train_set_df=Train_set_df[selected_features]
Test_set_df=Test_set_df[selected_features]
Val_set_df=Val_set_df[selected_features]

In [193]:
Train_set_df['Label'] = (Train_set_df['positive_blood_culture'] | Train_set_df['positive_blood_culture_in_week'])
Test_set_df['Label'] = (Test_set_df['positive_blood_culture'] | Test_set_df['positive_blood_culture_in_week'])
Val_set_df['Label'] = (Val_set_df['positive_blood_culture'] | Val_set_df['positive_blood_culture_in_week'])

X_train = Train_set_df.drop(columns=['Label','positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','datapoint','Line_Presense'])#
y_train = Train_set_df['Label']

# Prepare the test data
X_test = Test_set_df.drop(columns=['Label','positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','datapoint','Line_Presense'])#
y_test = Test_set_df['Label']


X_val = Val_set_df.drop(columns=['Label','positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','datapoint','Line_Presense'])#
y_val = Val_set_df['Label']

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_97924/4149568682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Train_set_df['Label'] = (Train_set_df['positive_blood_culture'] | Train_set_df['positive_blood_culture_in_week'])


In [194]:
X_train = X_train.dropna(axis=1, how='all')
X_val = X_val[X_train.columns]
X_test = X_test[X_train.columns]

In [195]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
imputer.fit(X_train)  
X_train = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
X_val = pd.DataFrame(imputer.transform(X_val), columns=X_val.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


In [196]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit the scaler on the training data
scaler.fit(X_train)

# Transform the training, test, and validation data
X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)
X_val = pd.DataFrame(scaler.transform(X_val), columns=X_train.columns)

In [197]:
y_train.unique()

<IntegerArray>
[0, 1]
Length: 2, dtype: Int64

In [198]:
from sklearn.utils.class_weight import compute_class_weight
# Compute class weights 

# Count number of samples per class
"""
counts = pd.Series(y_train).value_counts().to_dict()

# Total samples
total = len(y_train)

# Manually compute weight for each class (e.g., scale positive more heavily)
class_weights = {
    0: 1.0,  # keep negative as baseline
    1: total / (2 * counts[1])  # scale positive inversely proportional to its count
}
"""
# for higher sensitivity avoid scaling posetive cases
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.array([0, 1]),
    y=y_train 
)


In [199]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
import numpy as np


# Map into dict
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}


model_l2 = LogisticRegression(max_iter=10000,class_weight=class_weight_dict, C=0.01, penalty='l2',solver='lbfgs')
model_l2.fit(X_train, y_train)

# Predict probabilities for the test set
y_pred_prob_l2 = model_l2.predict_proba(X_test)[:, 1]

# Calculate the AUC
auc_l2 = roc_auc_score(y_test, y_pred_prob_l2)
print(f"AUC for the test set with L2 regularization: {auc_l2:.4f}")


# Calculate the AUC
yt_pred_prob_l2 = model_l2.predict_proba(X_val)[:, 1]
auc_l2 = roc_auc_score(y_val, yt_pred_prob_l2)
print(f"AUC for the val set with L2 regularization: {auc_l2:.4f}")

AUC for the test set with L2 regularization: 0.7329
AUC for the train set with L2 regularization: 0.7572


In [200]:
import numpy as np
from sklearn.metrics import roc_curve, confusion_matrix

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_l2)

# Find the threshold where sensitivity (TPR) >= 0.9
threshold_index = np.argmax(tpr >= 0.9)
optimal_threshold = thresholds[threshold_index]

# Use the optimal threshold to make binary predictions
y_pred_optimal = (y_pred_prob_l2 >= optimal_threshold).astype(int)

# Calculate the confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_optimal).ravel()

# Calculate sensitivity, specificity, PPV, and NPV
sensitivity = tp / (tp + fn)  # Sensitivity or Recall
specificity = tn / (tn + fp)  # Specificity
ppv = tp / (tp + fp)          # Positive Predictive Value (Precision)
npv = tn / (tn + fn)          # Negative Predictive Value

print(f"Optimal Threshold: {optimal_threshold:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"PPV (Precision): {ppv:.4f}")
print(f"NPV: {npv:.4f}")

Optimal Threshold: 0.4066
Sensitivity: 0.9074
Specificity: 0.3618
PPV (Precision): 0.0439
NPV: 0.9918


In [202]:
from sklearn.utils import resample
y_pred_prob =  model_l2.predict_proba(X_test) [:,1]


# Step 2: Bootstrap resampling
n_iterations = 1000  # Number of bootstrap samples
auc_scores = []

for _ in range(n_iterations):
    # Resample the data with replacement (X_test and y_test)
    X_resampled, y_resampled = resample(X_test, y_test, replace=True)
       
    # Get the predicted probabilities for the resampled data
    y_pred_prob_resampled =  model_l2.predict_proba(X_resampled)[:,1]
    
    # Calculate the AUC score for the resampled data
    auc = roc_auc_score(y_resampled, y_pred_prob_resampled)
    auc_scores.append(auc)

# Step 3: Calculate the 95% confidence interval for AUC
ci_lower = np.percentile(auc_scores, 2.5)
ci_upper = np.percentile(auc_scores, 97.5)

print(f"95% confidence interval for AUC: ({ci_lower}, {ci_upper})")

95% confidence interval for AUC: (0.6844405769013293, 0.7786331324344548)


# Pointing system

In [203]:
import numpy as np

# Get coefficients and feature names
coefficients = model_l2.coef_[0]
feature_names = X_train.columns

# 1. Find nonzero coefficients
non_zero_indices = np.where(coefficients != 0)[0]
non_zero_features = feature_names[non_zero_indices]
non_zero_coefficients = coefficients[non_zero_indices]

# 2. Sort by absolute value of coefficients
sorted_indices = np.argsort(-np.abs(non_zero_coefficients))  # negative sign for descending sort
sorted_features = non_zero_features[sorted_indices]
sorted_coefficients = non_zero_coefficients[sorted_indices]

# 3. Print sorted features and coefficients
for feature, coef in zip(sorted_features, sorted_coefficients):
    print(f"Feature: {feature}, Coefficient: {coef:.4f}")

Feature: median_procalcitonin, Coefficient: 0.6050
Feature: bmi, Coefficient: -0.4810
Feature: age, Coefficient: 0.2644
Feature: min_hgb, Coefficient: -0.2125
Feature: max_heartrate, Coefficient: 0.1852
Feature: avg_neutrophils, Coefficient: 0.1634
Feature: median_bun, Coefficient: 0.1507
Feature: max_na, Coefficient: -0.1102
Feature: avg_lymphocytes, Coefficient: -0.0980
Feature: WBC_urine, Coefficient: 0.0909
Feature: min_resprate, Coefficient: 0.0816
Feature: min_temp, Coefficient: 0.0710
Feature: Bacteria_urine, Coefficient: 0.0598
Feature: ET_Tube, Coefficient: 0.0587
Feature: Leukocyte_Esterase, Coefficient: 0.0494
Feature: max_hco3, Coefficient: 0.0239
Feature: avg_spo2, Coefficient: 0.0041
Feature: min_wbc, Coefficient: -0.0009
Feature: min_diasbp, Coefficient: -0.0009
Feature: max_cr, Coefficient: 0.0008


In [204]:
import numpy as np

# Get original coefficients and intercept
coefficients = model_l2.coef_[0]
intercept = model_l2.intercept_[0]
feature_names = X_train.columns  # or replace with your feature list

# Round the coefficients
rounded_coefficients = np.round(coefficients, 2)

# Filter: keep only coefficients that are non-zero **after rounding**
non_zero_mask = rounded_coefficients != 0
non_zero_coefficients = rounded_coefficients[non_zero_mask]
non_zero_features = feature_names[non_zero_mask]

# Round intercept
rounded_intercept = np.round(intercept, 2)

# Build the logistic regression equation
equation_terms = [f"{coef} * {name}" for coef, name in zip(non_zero_coefficients, non_zero_features)]
equation = " + ".join(equation_terms)
full_equation = f"logit(P) = {rounded_intercept} + {equation}"

# Print the equation
print("Logistic Regression Equation:")
print(full_equation)

Logistic Regression Equation:
logit(P) = -0.16 + -0.48 * bmi + 0.16 * avg_neutrophils + 0.26 * age + -0.11 * max_na + 0.19 * max_heartrate + 0.61 * median_procalcitonin + 0.07 * min_temp + -0.1 * avg_lymphocytes + 0.08 * min_resprate + 0.02 * max_hco3 + -0.21 * min_hgb + 0.15 * median_bun + 0.06 * ET_Tube + 0.09 * WBC_urine + 0.06 * Bacteria_urine + 0.05 * Leukocyte_Esterase


In [205]:
import numpy as np

# Get original coefficients and intercept
coefficients = model_l2.coef_[0]
intercept = model_l2.intercept_[0]
feature_names = X_train.columns

# Round the coefficients
rounded_coefficients = np.round(coefficients, 2)

# Filter: keep only coefficients that are non-zero **after rounding**
non_zero_mask = rounded_coefficients != 0
non_zero_coefficients = rounded_coefficients[non_zero_mask]
non_zero_features = feature_names[non_zero_mask]

# Now select only the top 6 features
selected_features = [
    'bmi', 
    'avg_neutrophils', 
    'age', 
    'max_na', 
    'max_heartrate', 
    'median_procalcitonin', 
    'min_temp', 
    'avg_lymphocytes', 
    'min_diasbp', 
    'avg_spo2', 
    'min_resprate', 
    'max_hco3', 
    'min_wbc', 
    'min_hgb', 
    'median_bun', 
    'max_cr', 
    'ET_Tube', 
    'WBC_urine', 
    'Bacteria_urine', 
    'Leukocyte_Esterase'
]

# Mask for selected features
selected_mask = np.isin(non_zero_features, selected_features)

selected_coefficients = non_zero_coefficients[selected_mask]
selected_feature_names = non_zero_features[selected_mask]

# Round intercept
rounded_intercept = np.round(intercept, 2)

# Build the logistic regression equation
equation_terms = [f"{coef} * {name}" for coef, name in zip(selected_coefficients, selected_feature_names)]
equation = " + ".join(equation_terms)
full_equation = f"logit(P) = {rounded_intercept} + {equation}"

print("Logistic Regression Equation:")
print(full_equation)

Logistic Regression Equation:
logit(P) = -0.16 + -0.48 * bmi + 0.16 * avg_neutrophils + 0.26 * age + -0.11 * max_na + 0.19 * max_heartrate + 0.61 * median_procalcitonin + 0.07 * min_temp + -0.1 * avg_lymphocytes + 0.08 * min_resprate + 0.02 * max_hco3 + -0.21 * min_hgb + 0.15 * median_bun + 0.06 * ET_Tube + 0.09 * WBC_urine + 0.06 * Bacteria_urine + 0.05 * Leukocyte_Esterase


In [208]:
selected_features = [
    'bmi', 'avg_neutrophils', 'age', 'max_na', 'max_heartrate', 'median_procalcitonin', 
    'min_temp', 'avg_lymphocytes', 'min_diasbp', 'avg_spo2', 'min_resprate', 'max_hco3', 
    'min_wbc', 'min_hgb', 'median_bun', 'max_cr', 'ET_Tube', 'WBC_urine', 'Bacteria_urine', 'Leukocyte_Esterase'
]

coefficients = np.array([
    -0.4664, 0.3382, 0.2440, -0.2402, 0.2359, 0.2209, 0.2194, -0.2002,
    0.1982, -0.1810, 0.1427, -0.1357, 0.0863, -0.0963, 0.0579, -0.0391,
    0.0544, 0.0896, 0.0542, 0.0579
])

# 1. Get scaler mean and std for each feature
feature_means = pd.Series(scaler.mean_, index=scaler.feature_names_in_)
feature_stds = pd.Series(np.sqrt(scaler.var_), index=scaler.feature_names_in_)

# 2. Median (standardized space) from X_train
baseline_standardized = X_train[selected_features].median()

# 3. Reverse standardization
baseline_real = baseline_standardized * feature_stds[selected_features] + feature_means[selected_features]

#4.
points_per_feature = np.abs(coefficients)


#5 Build the new pointing table
points_table = pd.DataFrame({
    'Feature': selected_features,
    'Coefficient': coefficients,
    'Baseline (real units)': baseline_real.round(2),
    'Points per Unit': np.abs(coefficients).round(2)
})

# Step 2: Display it sorted
points_table = points_table.sort_values(by='Points per Unit', ascending=False)

print(points_table)


                                   Feature  Coefficient  \
bmi                                    bmi      -0.4664   
avg_neutrophils            avg_neutrophils       0.3382   
age                                    age       0.2440   
max_na                              max_na      -0.2402   
max_heartrate                max_heartrate       0.2359   
median_procalcitonin  median_procalcitonin       0.2209   
min_temp                          min_temp       0.2194   
avg_lymphocytes            avg_lymphocytes      -0.2002   
min_diasbp                      min_diasbp       0.1982   
avg_spo2                          avg_spo2      -0.1810   
max_hco3                          max_hco3      -0.1357   
min_resprate                  min_resprate       0.1427   
min_hgb                            min_hgb      -0.0963   
min_wbc                            min_wbc       0.0863   
WBC_urine                        WBC_urine       0.0896   
median_bun                      median_bun       0.0579 

In [209]:
selected_features = [
    'bmi', 'avg_neutrophils', 'age', 'max_na', 'max_heartrate', 'median_procalcitonin', 
    'min_temp', 'avg_lymphocytes', 'min_diasbp', 'avg_spo2', 'min_resprate', 'max_hco3', 
    'min_wbc', 'min_hgb', 'median_bun', 'max_cr', 'ET_Tube', 'WBC_urine', 'Bacteria_urine', 'Leukocyte_Esterase'
]


def calculate_patient_points(patient):
    score = 0
    
    if patient['bmi'] < 18.4:
        score += 5
    if patient['avg_neutrophils'] > 21.5:
        score += 3
    if patient['age'] > 5:
        score += 2
    if patient['max_na'] < 136.0:
        score += 2
    if patient['max_heartrate'] > 140.0:
        score += 2
    if patient['median_procalcitonin'] > 0.22:
        score += 2
    if patient['min_temp'] > 99.1:
        score += 2
    if patient['avg_lymphocytes'] < 32.0:
        score += 2
    if patient['min_diasbp'] > 65.0:
        score += 2
    if patient['avg_spo2'] < 98.7:
        score += 2
    if patient['max_hco3'] < 23.0:
        score += 1
    if patient['min_resprate'] > 22.0:
        score += 1
    if patient['min_hgb'] < 11500:
        score += 1
    if patient['min_wbc'] > 9.1:
        score += 1
    if patient['WBC_urine'] == 1:
        score += 1
    if patient['median_bun'] > 11:
        score += 1
    if patient['Leukocyte_Esterase'] == 1:
        score += 1
    if patient['ET_Tube'] == 1:
        score += 1
    if patient['Bacteria_urine'] == 1:
        score += 1
    
    return score

X_test_real = X_test[selected_features] * feature_stds[selected_features] + feature_means[selected_features]
scores = X_test_real[selected_features].apply(calculate_patient_points, axis=1)
min_score = scores.min()
max_score = scores.max()
print(min_score,max_score)


4 28


In [217]:
shifted_scores = scores - min_score  # All scores now ≥ 0
thresholds = np.arange(0, np.ceil(shifted_scores.max()) + 1)
results = []

auc = roc_auc_score(y_test, scores)
print(f"AUC: {auc:.3f}")

for t in thresholds:
    y_pred = (shifted_scores >= t).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    results.append({
        'Threshold': t,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'PPV': ppv,
        'NPV': npv,
        'Accuracy': accuracy
    })


results

AUC: 0.641


[{'Threshold': 0.0,
  'Sensitivity': 1.0,
  'Specificity': 0.0,
  'PPV': 0.031286210892236384,
  'NPV': 0,
  'Accuracy': 0.031286210892236384},
 {'Threshold': 1.0,
  'Sensitivity': 1.0,
  'Specificity': 0.00687799043062201,
  'PPV': 0.031496062992125984,
  'NPV': 1.0,
  'Accuracy': 0.03794901506373117},
 {'Threshold': 2.0,
  'Sensitivity': 0.9814814814814815,
  'Specificity': 0.020035885167464115,
  'PPV': 0.03133313626958321,
  'NPV': 0.9710144927536232,
  'Accuracy': 0.05011587485515643},
 {'Threshold': 3.0,
  'Sensitivity': 0.9814814814814815,
  'Specificity': 0.04724880382775119,
  'PPV': 0.03219927095990279,
  'NPV': 0.9875,
  'Accuracy': 0.07647740440324449},
 {'Threshold': 4.0,
  'Sensitivity': 0.9814814814814815,
  'Specificity': 0.07954545454545454,
  'PPV': 0.03329145728643216,
  'NPV': 0.9925373134328358,
  'Accuracy': 0.10776361529548088},
 {'Threshold': 5.0,
  'Sensitivity': 0.9351851851851852,
  'Specificity': 0.11931818181818182,
  'PPV': 0.03315824031516743,
  'NPV': 0.

In [218]:
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
import numpy as np

# Step 1: Assume you already have `scores` from your custom point function and `y_test`
n_iterations = 1000
auc_scores = []

# Combine scores and labels for easier resampling
score_label_df = pd.DataFrame({
    'score': scores,
    'label': y_test
})

score_label_df = score_label_df.dropna()

# Step 2: Bootstrap
for _ in range(n_iterations):
    sample = score_label_df.sample(frac=1, replace=True)
    auc = roc_auc_score(sample['label'], sample['score'])
    auc_scores.append(auc)

# Step 3: 95% CI
ci_lower = np.percentile(auc_scores, 2.5)
ci_upper = np.percentile(auc_scores, 97.5)

print(f"95% Confidence Interval for AUC: ({ci_lower:.3f}, {ci_upper:.3f})")

95% Confidence Interval for AUC: (0.497, 0.763)


# XGboost

In [77]:
Train_set_df['Label'] = (Train_set_df['positive_blood_culture'] | Train_set_df['positive_blood_culture_in_week'])
Test_set_df['Label'] = (Test_set_df['positive_blood_culture'] | Test_set_df['positive_blood_culture_in_week'])
Val_set_df['Label'] = (Val_set_df['positive_blood_culture'] | Val_set_df['positive_blood_culture_in_week'])

X_train = Train_set_df.drop(columns=['Label','positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','datapoint','Line_Presense'])
y_train = Train_set_df['Label']

# Prepare the test data
X_test = Test_set_df.drop(columns=['Label','positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','datapoint','Line_Presense'])
y_test = Test_set_df['Label']


X_val = Val_set_df.drop(columns=['Label','positive_blood_culture', 'positive_blood_culture_in_week','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','datapoint','Line_Presense'])
y_val = Val_set_df['Label']

In [78]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
dval = xgb.DMatrix(X_val, label=y_val)

In [88]:
# Calculate the ratio of negative to positive samples
scale_pos_weight =(len(y_train[y_train == 0]) / len(y_train[y_train == 1]))

params = {
    'max_depth':2,
    'eta': 0.3,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'scale_pos_weight': scale_pos_weight  # Add this line
}
num_rounds = 30

In [89]:
bst = xgb.train(params, dtrain, num_rounds, evals=[(dval, 'val')])

[0]	val-auc:0.71451
[1]	val-auc:0.71437
[2]	val-auc:0.71647
[3]	val-auc:0.71355
[4]	val-auc:0.74924
[5]	val-auc:0.75233
[6]	val-auc:0.76392
[7]	val-auc:0.77518
[8]	val-auc:0.77896
[9]	val-auc:0.78125
[10]	val-auc:0.78357
[11]	val-auc:0.79237
[12]	val-auc:0.79745
[13]	val-auc:0.80262
[14]	val-auc:0.80293
[15]	val-auc:0.80144
[16]	val-auc:0.80132
[17]	val-auc:0.80238
[18]	val-auc:0.80399
[19]	val-auc:0.78944
[20]	val-auc:0.79625
[21]	val-auc:0.79763
[22]	val-auc:0.80304
[23]	val-auc:0.80281
[24]	val-auc:0.79709
[25]	val-auc:0.80159
[26]	val-auc:0.80184
[27]	val-auc:0.80940
[28]	val-auc:0.80156
[29]	val-auc:0.80353


In [90]:
y_pred = bst.predict(dtest)
y_pred_binary = (y_pred > 0.5).astype(int)  # converting probabilities to binary output
y_pred_val = bst.predict(dval)

In [91]:
from sklearn.metrics import accuracy_score, roc_auc_score
auc_score = roc_auc_score(y_test, y_pred)
print(f'AUC Score: {auc_score:.2f}')

AUC Score: 0.71


In [215]:
y_pred_prob = bst.predict(dtest, output_margin=False)  # returns probabilities for each class

# Step 2: Bootstrap resampling
n_iterations = 1000  # Number of bootstrap samples
auc_scores = []

for _ in range(n_iterations):
    # Resample the data with replacement (X_test and y_test)
    X_resampled, y_resampled = resample(X_test, y_test, replace=True)
    
    # Convert the resampled features back to a DMatrix
    dtest_resampled = xgb.DMatrix(X_resampled)  # Resample data needs to be in DMatrix format
    
    # Get the predicted probabilities for the resampled data
    y_pred_prob_resampled = bst.predict(dtest_resampled, output_margin=False)
    
    # Calculate the AUC score for the resampled data
    auc = roc_auc_score(y_resampled, y_pred_prob_resampled)
    auc_scores.append(auc)

# Step 3: Calculate the 95% confidence interval for AUC
ci_lower = np.percentile(auc_scores, 2.5)
ci_upper = np.percentile(auc_scores, 97.5)

print(f"95% confidence interval for AUC: ({ci_lower}, {ci_upper})")

95% confidence interval for AUC: (0.6552451434160992, 0.7533567366972678)


In [218]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

In [219]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)  # Calculate area under the curve

fpr_val, tpr_val,_ = roc_curve(y_val, y_pred_val)
roc_auc_tr = auc(fpr_val, tpr_val)

In [221]:
index=np.where(tpr>=0.9)[0][0]
print('sensitivity',tpr[index])
print('specifity',1-fpr[index])
from sklearn.metrics import confusion_matrix, precision_score, recall_score
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
recall_val = recall_score(y_test, y_pred_binary)
print('Threshod',thresholds[index])
ppv_val = precision_score(y_test, y_pred_binary)
npv_val = tn / (tn + fn)
specificity_val = tn / (tn + fp)
print(f"Recall (Sensitivity): {recall_val:.2f}")
print(f"specificity: {specificity_val:.2f}")
print(f"Precision (PPV): {ppv_val:.2f}")
print(f"Negative Predictive Value (NPV): {npv_val:.2f}")


sensitivity 0.9166666666666666
specifity 0.32894736842105265
Threshod 0.29400873
Recall (Sensitivity): 0.90
specificity: 0.35
Precision (PPV): 0.04
Negative Predictive Value (NPV): 0.99


# Adding Notes

In [223]:
%%bigquery Train_set 
select anon_id,
pat_enc_csn_id_coded,
order_proc_id_coded,
notedatetime,
deid_note_text 
from som-nero-phi-jonc101.PEDsblood_culture_stewardship.Notes
where Extract(year from notedatetime)<2022

Query is running:   0%|          |

Downloading:   0%|          |

In [226]:
%%bigquery Val_set 
select anon_id,
pat_enc_csn_id_coded,
order_proc_id_coded,
notedatetime,
deid_note_text 
from som-nero-phi-jonc101.PEDsblood_culture_stewardship.Notes
where  Extract(year from notedatetime)>=2022 and 
Extract(year from notedatetime)<2023

Query is running:   0%|          |

Downloading:   0%|          |

In [227]:
%%bigquery Test_set 
select anon_id,
pat_enc_csn_id_coded,
order_proc_id_coded,
notedatetime,
deid_note_text 
from som-nero-phi-jonc101.PEDsblood_culture_stewardship.Notes
where  
Extract(year from notedatetime)>=2023

Query is running:   0%|          |

Downloading:   0%|          |

In [228]:
Train_set.to_csv('Train_set.csv',index=False)
Val_set.to_csv('Val_set.csv',index=False)
Test_set.to_csv('Test_set.csv',index=False)

# perfom Modeling:

In [12]:
## read the embeding 
train_set_embeding=pd.read_csv('Train_set_with_embeddings.csv')
train_set_embeding2=pd.read_csv('Train_set_with_embeddings_part2.csv')
train_set_embeding=pd.concat([train_set_embeding,train_set_embeding2],axis=0)
train_set_embeding.drop_duplicates(subset=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','notedatetime','deid_note_text'],inplace=True)
train_set_embeding.shape


test_set_embeding=pd.read_csv('Test_set_with_embeddings.csv')
test_set_embeding.drop_duplicates(subset=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','notedatetime','deid_note_text'],inplace=True)
test_set_embeding.shape


val_set_embeding=pd.read_csv('Val_set_with_embeddings.csv')
val_set_embeding.drop_duplicates(subset=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','notedatetime','deid_note_text'],inplace=True)
val_set_embeding.shape


(6090, 6)

In [13]:
train_set_embeding[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','notedatetime','deid_note_text']].drop_duplicates().shape # unique orders 20056, unique notes = 39909,

(40638, 5)

In [14]:
Train_set_Notes=pd.read_csv('Train_set.csv')
Train_set_Notes=pd.merge(Train_set_Notes,train_set_embeding,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','notedatetime','deid_note_text'])
Train_set_Notes=train_set_embeding

Test_set_Notes=pd.read_csv('Test_set.csv')
Test_set_Notes=pd.merge(Test_set_Notes,test_set_embeding,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','notedatetime','deid_note_text'])
Test_set_Notes=test_set_embeding

Val_set_Notes=pd.read_csv('Val_set.csv')
Val_set_Notes=pd.merge(Val_set_Notes,val_set_embeding,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','notedatetime','deid_note_text'])
Val_set_Notes=val_set_embeding

In [15]:
Train_set_df_temp=Train_set_df[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','blood_culture_order_datetime']]
Train_set_df_temp.drop_duplicates(inplace=True)
Train_set_Notes=pd.merge(Train_set_Notes,Train_set_df_temp,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])


Test_set_df_temp=Test_set_df[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','blood_culture_order_datetime']]
Test_set_df_temp.drop_duplicates(inplace=True)
Test_set_Notes=pd.merge(Test_set_Notes,Test_set_df_temp,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Test_set_Notes.shape


Val_set_df_temp=Val_set_df[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','blood_culture_order_datetime']]
Val_set_df_temp.drop_duplicates(inplace=True)
Val_set_Notes=pd.merge(Val_set_Notes,Val_set_df_temp,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Val_set_Notes.shape

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_34790/3777328094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Train_set_df_temp.drop_duplicates(inplace=True)
/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_34790/3777328094.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_set_df_temp.drop_duplicates(inplace=True)
/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_34790/3777328094.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

(6090, 7)

In [16]:
Train_set_Notes['notedatetime'] = pd.to_datetime(Train_set_Notes['notedatetime'])
Train_set_Notes['blood_culture_order_datetime'] = pd.to_datetime(Train_set_Notes['blood_culture_order_datetime'])
Train_set_Notes['Notetime_cultrueTime'] = (Train_set_Notes['blood_culture_order_datetime'] - Train_set_Notes['notedatetime']).dt.total_seconds()/3600
Train_set_Notes=Train_set_Notes[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','Notetime_cultrueTime','sentence_qmbedding']]


Test_set_Notes['notedatetime'] = pd.to_datetime(Test_set_Notes['notedatetime'])
Test_set_Notes['blood_culture_order_datetime'] = pd.to_datetime(Test_set_Notes['blood_culture_order_datetime'])
Test_set_Notes['Notetime_cultrueTime'] = (Test_set_Notes['blood_culture_order_datetime'] - Test_set_Notes['notedatetime']).dt.total_seconds()/3600
Test_set_Notes=Test_set_Notes[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','Notetime_cultrueTime','sentence_qmbedding']]



Val_set_Notes['notedatetime'] = pd.to_datetime(Val_set_Notes['notedatetime'])
Val_set_Notes['blood_culture_order_datetime'] = pd.to_datetime(Val_set_Notes['blood_culture_order_datetime'])
Val_set_Notes['Notetime_cultrueTime'] = (Val_set_Notes['blood_culture_order_datetime'] - Val_set_Notes['notedatetime']).dt.total_seconds()/3600
Val_set_Notes=Val_set_Notes[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','Notetime_cultrueTime','sentence_qmbedding']]

In [17]:
min_idx = Train_set_Notes.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'])['Notetime_cultrueTime'].idxmin()
Train_set_Notes = Train_set_Notes.loc[min_idx].reset_index(drop=True)
Train_set_Notes.shape


min_idx = Test_set_Notes.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'])['Notetime_cultrueTime'].idxmin()
Test_set_Notes = Test_set_Notes.loc[min_idx].reset_index(drop=True)
Test_set_Notes.shape


min_idx = Val_set_Notes.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'])['Notetime_cultrueTime'].idxmin()
Val_set_Notes = Val_set_Notes.loc[min_idx].reset_index(drop=True)
Val_set_Notes.shape

(3016, 5)

In [18]:
Train_set_df['Label'] = (Train_set_df['positive_blood_culture'] | Train_set_df['positive_blood_culture_in_week'])
Test_set_df['Label'] = (Test_set_df['positive_blood_culture'] | Test_set_df['positive_blood_culture_in_week'])
Val_set_df['Label'] = (Val_set_df['positive_blood_culture'] | Val_set_df['positive_blood_culture_in_week'])

In [19]:
Train_set_df=pd.merge(Train_set_Notes,Train_set_df,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Train_set_df.drop_duplicates(subset=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'],inplace=True)
Train_set_df['datapoint'] = Train_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1


Test_set_df=pd.merge(Test_set_Notes,Test_set_df,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Test_set_df.drop_duplicates(subset=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'],inplace=True)
Test_set_df['datapoint'] = Train_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1

Val_set_df=pd.merge(Val_set_Notes,Val_set_df,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Val_set_df.drop_duplicates(subset=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'],inplace=True)
Val_set_df['datapoint'] = Val_set_df.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded']).ngroup() + 1


In [20]:
Train_set_df['birth_date_jittered'] = pd.to_datetime(Train_set_df['birth_date_jittered'])
Train_set_df['blood_culture_order_datetime'] = pd.to_datetime(Train_set_df['blood_culture_order_datetime'])
Train_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Train_set_df['earliest_iv_antibiotic_datetime'])

Val_set_df['birth_date_jittered'] = pd.to_datetime(Val_set_df['birth_date_jittered'])
Val_set_df['blood_culture_order_datetime'] = pd.to_datetime(Val_set_df['blood_culture_order_datetime'])
Val_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Val_set_df['earliest_iv_antibiotic_datetime'])


Test_set_df['birth_date_jittered'] = pd.to_datetime(Test_set_df['birth_date_jittered'])
Test_set_df['blood_culture_order_datetime'] = pd.to_datetime(Test_set_df['blood_culture_order_datetime'])
Test_set_df['earliest_iv_antibiotic_datetime'] = pd.to_datetime(Test_set_df['earliest_iv_antibiotic_datetime'])


# Calculate the difference in hours:
Train_set_df['age'] = (Train_set_df['blood_culture_order_datetime'] - Train_set_df['birth_date_jittered']).dt.days 
Test_set_df['age'] = (Test_set_df['blood_culture_order_datetime'] - Test_set_df['birth_date_jittered']).dt.days #/ 3600
Val_set_df['age'] = (Val_set_df['blood_culture_order_datetime'] - Val_set_df['birth_date_jittered']).dt.days #/ 3600

Train_set_df['hours_between_cult_abx'] = (Train_set_df['blood_culture_order_datetime'] - Train_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600
Test_set_df['hours_between_cult_abx'] = (Test_set_df['blood_culture_order_datetime'] - Test_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600
Val_set_df['hours_between_cult_abx'] = (Val_set_df['blood_culture_order_datetime'] - Val_set_df['earliest_iv_antibiotic_datetime']).dt.total_seconds() / 3600


In [21]:
Train_set_df['gender'] = Train_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))
Test_set_df['gender'] = Test_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))
Val_set_df['gender'] = Val_set_df['gender'].apply(lambda x: 1 if x == 'Male' else (0 if x == 'Female' else None))


Train_set_df['Leukocyte_Esterase'] = Train_set_df['Leukocyte_Esterase'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Test_set_df['Leukocyte_Esterase'] = Test_set_df['Leukocyte_Esterase'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Val_set_df['Leukocyte_Esterase'] = Val_set_df['Leukocyte_Esterase'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))


Train_set_df['WBC_urine'] = Train_set_df['WBC_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Test_set_df['WBC_urine'] = Test_set_df['WBC_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Val_set_df['WBC_urine'] = Val_set_df['WBC_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))


Train_set_df['Bacteria_urine'] = Train_set_df['Bacteria_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Test_set_df['Bacteria_urine'] = Test_set_df['Bacteria_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Val_set_df['Bacteria_urine'] = Val_set_df['Bacteria_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))


Train_set_df['Nitrite_urine'] = Train_set_df['Nitrite_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Test_set_df['Nitrite_urine'] = Test_set_df['Nitrite_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))
Val_set_df['Nitrite_urine'] = Val_set_df['Nitrite_urine'].apply(lambda x: 1 if x == 'POSITIVE' else (0 if x == 'NEGATIVE' else None))


Train_set_df['ET_Tube'] = Train_set_df['Line_Presense'].apply(lambda x: 1 if x == 'ET_Tube' else 0)
Test_set_df['ET_Tube'] = Test_set_df['Line_Presense'].apply(lambda x: 1 if x == 'ET_Tube' else 0 )
Val_set_df['ET_Tube'] = Val_set_df['Line_Presense'].apply(lambda x: 1 if x == 'ET_Tube' else 0)



Train_set_df['otherline'] = Train_set_df['Line_Presense'].apply(lambda x: 1 if x == 'otherline' else 0)
Test_set_df['otherline'] = Test_set_df['Line_Presense'].apply(lambda x: 1 if x == 'otherline' else 0 )
Val_set_df['otherline'] = Val_set_df['Line_Presense'].apply(lambda x: 1 if x == 'otherline' else 0)


In [22]:
# Feature Set
Identifiers=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','datapoint']
Labels=['Label']
Labs=['min_heartrate','max_heartrate', 'avg_heartrate', 'median_heartrate',
       'min_resprate', 'max_resprate', 'avg_resprate', 'median_resprate',
       'min_temp', 'max_temp', 'avg_temp', 'median_temp', 'min_sysbp',
       'max_sysbp', 'avg_sysbp', 'median_sysbp', 'min_diasbp',
       'max_diasbp', 'avg_diasbp', 'median_diasbp', 'min_wbc', 'max_wbc',
       'avg_wbc', 'median_wbc', 'min_neutrophils', 'max_neutrophils',
       'avg_neutrophils', 'median_neutrophils', 'min_lymphocytes',
       'max_lymphocytes', 'avg_lymphocytes', 'median_lymphocytes',
       'min_hgb', 'max_hgb', 'avg_hgb', 'median_hgb', 'min_plt',
       'max_plt', 'avg_plt', 'median_plt', 'min_na', 'max_na', 'avg_na',
       'median_na', 'min_hco3', 'max_hco3', 'avg_hco3', 'median_hco3',
       'min_bun', 'max_bun', 'avg_bun', 'median_bun', 'min_cr', 'max_cr',
       'avg_cr', 'median_cr', 'min_lactate', 'max_lactate', 'avg_lactate',
       'median_lactate', 'min_procalcitonin', 'max_procalcitonin',
       'avg_procalcitonin', 'median_procalcitonin','Leukocyte_Esterase',
       'WBC_urine', 'Bacteria_urine','Nitrite_urine', 
      'min_anc', 'max_anc', 'avg_anc', 'median_anc',
       'min_alc', 'max_alc', 'avg_alc', 'median_alc', 'min_spo2',
       'max_spo2', 'avg_spo2', 'median_spo2']
Demos=[ 'gender','age','bmi']
ABX=['vanc', 'zosyn', 'vanc_zosyn', 'other_ABX']
Time_Varient_features=['hours_between_cult_abx']
Embeddings=['sentence_qmbedding']
Diagnosis= ['ET_Tube','otherline','Transplant']
# Include or Exclude Diagnosis 
        #bacteremia', 'septic_shock', 'infective_endocarditis',
      # 'septic_thrombophlebitis', 'vascular_graft_infection', 'CRBSI',
      # 'infectious_discitis', 'epidural_abscess', 'septic_arthritis',
      # 'meningitis', 'meningitis_bacteria', 'cholangitis',
      # 'bacterial_cholangitis', 'pyelonephritis',
      # 'acute_bacterial_pyelonephritis', 'severe_pneumonia',
      # 'acute_hematogenous_osteomyelitis', 'asplenia',
      # 'immunocompromised_state', 'severe_cellulitis', 'cystitis',
      # 'prostatitis', 'CAP', 'diabetic_foot_infection', 'colitis',
      # 'aspiration_pneumonia', 'uncomplicated_cholecystitis',
      # 'uncomplicated_diverticulitis', 'Uncomplicated_pancreatitis',
Feature_set=Identifiers+Labels+Labs+Demos+Diagnosis+Time_Varient_features+ABX+Embeddings
Train_set_df=Train_set_df[Feature_set]
Test_set_df=Test_set_df[Feature_set]
Val_set_df=Val_set_df[Feature_set]

In [23]:
import ast
Train_set_df['sentence_qmbedding'] = Train_set_df['sentence_qmbedding'].apply(ast.literal_eval)
Test_set_df['sentence_qmbedding'] = Test_set_df['sentence_qmbedding'].apply(ast.literal_eval)
Val_set_df['sentence_qmbedding'] = Val_set_df['sentence_qmbedding'].apply(ast.literal_eval)

In [58]:
Train_set_df = Train_set_df.reset_index(drop=True)
Test_set_df = Test_set_df.reset_index(drop=True)
Val_set_df = Val_set_df.reset_index(drop=True)

In [59]:
embedding_Trdf = pd.DataFrame(Train_set_df['sentence_qmbedding'].tolist(),
                               columns=[f'embedding_{i}' for i in range(len(Train_set_df['sentence_qmbedding'].iloc[0]))])

embedding_Tedf = pd.DataFrame(Test_set_df['sentence_qmbedding'].tolist(),
                               columns=[f'embedding_{i}' for i in range(len(Test_set_df['sentence_qmbedding'].iloc[0]))])

embedding_Valdf = pd.DataFrame(Val_set_df['sentence_qmbedding'].tolist(),
                               columns=[f'embedding_{i}' for i in range(len(Val_set_df['sentence_qmbedding'].iloc[0]))])


Train_set_df = pd.concat([Train_set_df.drop(columns=['sentence_qmbedding']), embedding_Trdf], axis=1)
Test_set_df = pd.concat([Test_set_df.drop(columns=['sentence_qmbedding']), embedding_Tedf], axis=1)
Val_set_df = pd.concat([Val_set_df.drop(columns=['sentence_qmbedding']), embedding_Valdf], axis=1)


In [67]:
X_train = Train_set_df.drop(columns=['Label','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','datapoint'])
y_train = Train_set_df['Label']

# Prepare the test data
X_test = Test_set_df.drop(columns=['Label','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','datapoint'])
y_test = Test_set_df['Label']


X_val = Val_set_df.drop(columns=['Label','anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','datapoint'])
y_val = Val_set_df['Label']

In [68]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
dval = xgb.DMatrix(X_val, label=y_val)

In [127]:
scale_pos_weight = (len(y_train[y_train == 0]) / len(y_train[y_train == 1]))
params = {
    'max_depth':3,
    'eta': 0.3,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'scale_pos_weight': scale_pos_weight  
}
num_rounds = 100

In [128]:
bst = xgb.train(params, dtrain, num_rounds, evals=[(dval, 'val')])

[0]	val-auc:0.66648
[1]	val-auc:0.66066
[2]	val-auc:0.70550
[3]	val-auc:0.71788
[4]	val-auc:0.72182
[5]	val-auc:0.72893
[6]	val-auc:0.73941
[7]	val-auc:0.73530
[8]	val-auc:0.74830
[9]	val-auc:0.73297
[10]	val-auc:0.73688
[11]	val-auc:0.75061
[12]	val-auc:0.74823
[13]	val-auc:0.75358
[14]	val-auc:0.76162
[15]	val-auc:0.77077
[16]	val-auc:0.77054
[17]	val-auc:0.76993
[18]	val-auc:0.77180
[19]	val-auc:0.76214
[20]	val-auc:0.76329
[21]	val-auc:0.76278
[22]	val-auc:0.76668
[23]	val-auc:0.76699
[24]	val-auc:0.76582
[25]	val-auc:0.77170
[26]	val-auc:0.77207
[27]	val-auc:0.77287
[28]	val-auc:0.77716
[29]	val-auc:0.77548
[30]	val-auc:0.77921
[31]	val-auc:0.78058
[32]	val-auc:0.78607
[33]	val-auc:0.78325
[34]	val-auc:0.77519
[35]	val-auc:0.77216
[36]	val-auc:0.77288
[37]	val-auc:0.77215
[38]	val-auc:0.77539
[39]	val-auc:0.77180
[40]	val-auc:0.77543
[41]	val-auc:0.77568
[42]	val-auc:0.77856
[43]	val-auc:0.77270
[44]	val-auc:0.77304
[45]	val-auc:0.76982
[46]	val-auc:0.76572
[47]	val-auc:0.76356
[4

In [129]:
y_pred = bst.predict(dtest)
y_pred_binary = (y_pred > 0.5).astype(int)  
y_pred_val = bst.predict(dval)

In [130]:
from sklearn.metrics import accuracy_score, roc_auc_score
auc_score = roc_auc_score(y_test, y_pred)
print(f'AUC Score: {auc_score:.2f}')


from sklearn.metrics import accuracy_score, roc_auc_score
val_auc_score = roc_auc_score(y_val, y_pred_val)
print(f' Val AUC Score: {val_auc_score:.2f}')

AUC Score: 0.80
 Val AUC Score: 0.78


In [ ]:
y_pred_prob = bst.predict(dtest, output_margin=False)  # returns probabilities for each class

# Step 2: Bootstrap resampling
n_iterations = 1000  # Number of bootstrap samples
auc_scores = []

for _ in range(n_iterations):
    # Resample the data with replacement (X_test and y_test)
    X_resampled, y_resampled = resample(X_test, y_test, replace=True)
    
    # Convert the resampled features back to a DMatrix
    dtest_resampled = xgb.DMatrix(X_resampled)  # Resample data needs to be in DMatrix format
    
    # Get the predicted probabilities for the resampled data
    y_pred_prob_resampled = bst.predict(dtest_resampled, output_margin=False)
    
    # Calculate the AUC score for the resampled data
    auc = roc_auc_score(y_resampled, y_pred_prob_resampled)
    auc_scores.append(auc)

# Step 3: Calculate the 95% confidence interval for AUC
ci_lower = np.percentile(auc_scores, 2.5)
ci_upper = np.percentile(auc_scores, 97.5)

print(f"95% confidence interval for AUC: ({ci_lower}, {ci_upper})")

In [132]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

In [133]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)  # Calculate area under the curve

fpr_val, tpr_val,_ = roc_curve(y_val, y_pred_val)
roc_auc_tr = auc(fpr_val, tpr_val)

In [134]:
index=np.where(tpr>=0.90)[0][0]
print('sensitivity',tpr[index])
print('specifity',1-fpr[index])
SEN=tpr[index]
SPEC=1-fpr[index]
P=y_test[y_test==1].shape[0]/y_test.shape[0]
PPV = (SEN * P) / (SEN * P + (1 - SPEC) * (1 - P))
NPV = (SPEC * (1 - P)) / ((1 - SEN) * P + SPEC * (1 - P))
print(PPV)
print(NPV)


sensitivity 0.9166666666666666
specifity 0.42985611510791366
0.04947526236881558
0.9937629937629938
